In [1]:
##Load the dataset

import pandas as pd

dataset=pd.read_csv('final_dataset_basicmlmodel.csv')
dataset.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [2]:
##Data Cleaning
##Remove special characters like #.,/<> etc. , noise like unicode characters â and ð, numerals and percentages

import re #regular expression library

def dataclean(text):
    text=re.sub(r'[^\x00-\x7F]+','',text)    ##remove all characters which are not in ASCII code 0 to 127 (7F)
    text=re.sub(r'[^a-zA-Z\']',' ',text)       ##remove all characters which is not a-z or A-Z
    text=text.lower()
    return text


dataset['clean_text']=dataset.tweet.apply(lambda x: dataclean(x))
dataset.head()

,id,label,tweet,clean_text
0,1,0,@user when a father is dysfunctional and is s...,user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...,user user thanks for lyft credit i can't us...
2,3,0,bihday your majesty,bihday your majesty
3,4,0,#model i love u take with u all the time in ...,model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation,factsguide society now motivation


In [3]:
###Feature engineering: creating numerical features to understand underlying text

##Define all functions to extract features like, number of words, number of neation words, number of prompt(question) words, 
##number of rare words in tweets 


from wordcloud import STOPWORDS

##Generate word frequency
def generate_freq(text):
    word_list=[]
    
    for word in text.split():
        word_list.extend(word)
    
    #Create word frequency using word list
    word_freq=pd.Series(word_list).value_counts()
    
    word_freq=word_freq.drop(STOPWORDS,errors='ignore') ## Drop the words which are STOPWORDS
    
    return word_freq

##check whether a negation term is present in the text
def check_neg(words):
    for word in words:
        if word in ['n','no','not','non'] or re.search(r"\wn't",word):
            return 1
    else:
            return 0

##check whether one of the 100 rare words are present in the text or not
def check_rare(words,rare_100):
    for word in words:
        if word in rare_100:
            return 1
    else:
            return 0

##check whether prompt words are present in it or not
def isquestion(words):
    for word in words:
        if word in ['when', 'what', 'how', 'why', 'who']:
            return 1
    else:
            return 0       

In [4]:
##Use the functions defined to extract features from the dataset and add columns to it

word_freq=generate_freq(dataset.clean_text.str)
#print(word_freq[-100:])
rare_100=word_freq[-100:]

#Number of words in a tweet
dataset['wordcount']=dataset.clean_text.str.split().apply(lambda x: len(x))
dataset['isneg']=dataset.clean_text.str.split().apply(lambda x: check_neg(x))
dataset['israre']=dataset.clean_text.str.split().apply(lambda x: check_rare(x,rare_100))
dataset['isprompt']=dataset.clean_text.str.split().apply(lambda x: isquestion(x))
dataset['charactercount']=dataset.clean_text.apply(lambda x: len(x))
#dataset
dataset.head()

,id,label,tweet,clean_text,wordcount,isneg,israre,isprompt,charactercount
0,1,0,@user when a father is dysfunctional and is s...,user when a father is dysfunctional and is s...,18,0,0,1,102
1,2,0,@user @user thanks for #lyft credit i can't us...,user user thanks for lyft credit i can't us...,19,1,1,0,122
2,3,0,bihday your majesty,bihday your majesty,3,0,0,0,21
3,4,0,#model i love u take with u all the time in ...,model i love u take with u all the time in ...,12,0,0,0,54
4,5,0,factsguide: society now #motivation,factsguide society now motivation,4,0,0,0,39


In [74]:
#### Splitting the dataset into train and test sets

from sklearn.model_selection import train_test_split

X=dataset[['wordcount','isneg','israre','isprompt','charactercount']]
y=dataset.label

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=27)

#print(X_train.head(),y_train.head())
#print(X_train.shape)
#print(y_train.shape)

##Since keras model expects input in the form of numpy array, we convert the features and labels in numpy array
X_train=np.array(X_train)
y_train=np.array(y_train)
X_test=np.array(X_test)
y_test=np.array(y_test)

In [75]:
#### Building a ML model for text classification
### We will use Keras Sequential NN model

import numpy as np
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.optimizers import Adam

#Initialize Sequential NN
model=Sequential([
Dense(64,activation='relu',input_shape=(5,)),
Dense(64,activation='relu'),
Dense(64,activation='sigmoid'),
Dense(2,activation='softmax')
]    
)

##Compiling the model
model.compile(
optimizer=Adam(lr=0.05),
loss='categorical_crossentropy',
metrics=['accuracy']
)

In [49]:
print(to_categorical(np.array(y_train)))

[[ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 ..., 
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]]


In [76]:
####Training the model to fit into the training set
from keras.utils import to_categorical

model.fit(
X_train,
to_categorical(y_train),
epochs=3,
batch_size=50,
validation_data=(X_test,to_categorical(y_test))
)

Train on 4717 samples, validate on 525 samples
Epoch 1/3
4717/4717 [==============================] - 0s - loss: 0.8395 - acc: 0.5194 - val_loss: 0.6815 - val_acc: 0.5905
Epoch 2/3
4717/4717 [==============================] - 0s - loss: 0.6976 - acc: 0.5457 - val_loss: 0.6941 - val_acc: 0.5905
Epoch 3/3
4717/4717 [==============================] - 0s - loss: 0.7053 - acc: 0.5427 - val_loss: 0.6908 - val_acc: 0.5905


In [81]:
##Evaluate the model with the test dataset

loss,accuracy=model.evaluate(
X_test,
to_categorical(y_test)
)


print("\nLoss in test data: ",loss)
print("Accuracy in test data: ",accuracy*100,"%")

 32/525 [>.............................] - ETA: 0s
Loss in test data:  0.690788102263
Accuracy in test data:  59.0476190533 %
